# Copy restacked WRF outputs to permanent storage (`/import/SNAP`)

This step will copy the `hourly/` and `daily/` subdirectories created from restacking, currently on scratch space, to `/import/SNAP` for storage and distribution. 

The strategy here will be to just give and `rsync` command for each subdirectory in the `hourly` and `daily` subdirectories, which should be all years of restacked data for a single variable for the WRF group being worked on. 

Run this cell to set up the environment for running the copy:

In [1]:
from multiprocessing import Pool
from config import *
import luts
import slurm

Create a subdirectory for the slurm scripts used for doing this copying:

In [2]:
slurm_copy_dir = slurm_dir.joinpath("copy_restacked")
slurm_copy_dir.mkdir(exist_ok=True)

Define a function to create the sbatch scripts for each of daily and hourly directories:

In [3]:
def wrap_write_sbatch(src):
    sbatch_fps = []
    for varname in luts.varnames:
        varname = varname.lower()
        # write to .slurm script
        sbatch_fp = slurm_copy_dir.joinpath(f"copy_restacked_{group}_{src.name}_{varname}.slurm")
        # filepath for slurm stdout
        sbatch_out_fp = slurm_copy_dir.joinpath(f"copy_restacked_{group}_{src.name}_{varname}_%j.out")
        sbatch_head = slurm.make_sbatch_head(
            slurm_email, partition, conda_init_script
        )

        args = {
            "sbatch_fp": sbatch_fp,
            "sbatch_out_fp": sbatch_out_fp,
            "src": src.joinpath(varname),
            "dst": output_dir.joinpath(src.name),
            "sbatch_head": sbatch_head,
        }

        slurm.write_sbatch_copy_restacked(**args)
        sbatch_fps.append(sbatch_fp)
    
    return sbatch_fps

Then create them:

In [4]:
sbatch_fps = wrap_write_sbatch(hourly_dir) + wrap_write_sbatch(daily_dir) 

And submit the `.slurm` scripts with `sbatch`:

In [5]:
job_ids = [slurm.submit_sbatch(fp) for fp in sbatch_fps]